# Unidad 3 - Actividad 1
# Materia: **Análisis de Algoritmos y Estructuras para Datos Masivos**
# Alumno: **Luis Fernando Izquierdo Berdugo**
# Fecha: **28 de Agosto de 2024**

**Objetivo**

Analizar y comparar la eficiencia de distintos algoritmos de búsqueda en listas de posteo, enfocándose en el análisis gráfico y la interpretación de los experimentos realizados.

**Instrucciones**
1. **Preparación de Datos**

- Utilice el archivo `listas-posteo-100.json`, que contiene las 100 listas de posteo más frecuentes en formato JSON. Trabaje solo con las listas, excluyendo los términos asociados.

- Asegúrese de no realizar copias mientras se manipulan los arreglos. Si se utiliza el ndarray o array se puede controlar la organización de memoria de los arreglos, también se puede asegurar que se manipulan solo valores numéricos nativos y no objetos de Python.

2. **Selección de Identificadores**

- Seleccione aleatoriamente 1000 identificadores de documentos, con valores entre 1 y n (donde n=50000).

3. **Experimentación y Medición**

- Mida el tiempo promedio de búsqueda para los 1000 identificadores en todas las listas (considerando 100×1000 operaciones).
- Determine la posición de inserción en cada búsqueda.
- Evalúe los siguientes algoritmos:
    - Búsqueda binaria acotada.
    - Búsqueda secuencial B_0​.
    - Búsqueda no acotada B_1.
    - Búsqueda no acotada B_2​.
    
**Nota:** Repita las búsquedas varias veces y promedielos para obtener mediciones más robustas.

4. **Análisis de Datos y Gráfica**

- Elabore una única gráfica que muestre los tiempos de ejecución en el eje Y y los algoritmos evaluados en el eje X.
- Realice un análisis detallado de esta gráfica, identificando patrones, diferencias y posibles explicaciones para el rendimiento de cada algoritmo.

5. **Desarrollo Teórico**

- Describa en pseudo-código la implementación de la búsqueda casi óptima *$B_k$*, explicando su lógica y funcionamiento

## Inciso 1 - Preparación de Datos
Lo primero será importar los módulos de Python a utilizar, en este caso serán `numpy`y `json`.

In [29]:
import json
import numpy as np

Debido a que el archivo no viene en el formato `json` común, se hizo un anális en el que se encontró que cada línea del archivo incluye la clave de la lista de posteo y los valores, por lo cual se desarrolló un código que analiza línea por línea para guardar los datos en un diccionario creado previamente llamado `valores`.

In [30]:
val = {}
with open('/Users/izluis/Documents/Maestria/Análisis de Algoritmos y Estructuras para Datos Masivos/Unidad 3/Unidad03-busqueda/datos/listas-posteo-100.json') as file:
  for line in file:
    data = json.loads(line)
    key, values = data
    val[key] = np.array(values)

## Inciso 2 - Selección de Identificadores

Se ejecuta un código sencillo para seleccionar 1000 valores aleatorios entre 1 y 5000, los cuales no se pueden repetir.

In [31]:
rango = np.arange(1, 5000 + 1)
ids = rango[np.random.choice(len(rango), size = 1000, replace=False)]

## Inciso 3 - Experimentación y Medición

In [32]:
import time
def times(function):
    total = 0
    positions = []
    for id in ids:
        start_time = time.time()
        for list in val.values():
            position = function(list, id)
            positions.append(position)
        final_time = time.time()
        total += final_time - start_time
    promedio = total / len(ids)*len(val)
    return promedio, positions

In [33]:
def bin_acot(lista, id):
    start = 0
    end = len(lista) -1
    while start <= end:
        middle = (start+end)//2
        if lista[middle] == id:
            return middle
        elif lista[middle] < id:
            start = middle+1
        else:
            end = middle - 1
    return end + 1
        

In [34]:
def sec(lista, id):
    for i, elemento in enumerate(lista):
        if elemento == id:
            return i
        elif elemento > id:
            return i
    # Si llegamos al final de la lista sin encontrar el elemento, se inserta al final
    return len(lista)

In [44]:
def bin_unb(lista, id):
    start = 0
    end = 1
    while id > lista[end]:
        end *= 2
    while start <= end:
        mid = (start + end) // 2
        if lista[mid] == id:
            return mid
        elif lista[mid] < id:
            start = mid + 1
        else:
            end = mid - 1
    return start
        

In [48]:
matrix = np.zeros((len(val), max(map(len, val.values()))), dtype=int)

for i, key in enumerate(val):
    matrix[i, :len(val[key])] = val[key]


[[    1     2     3 ... 49998 49999 50000]
 [    5     8    12 ...     0     0     0]
 [    2     3     7 ...     0     0     0]
 ...
 [  351   359   366 ...     0     0     0]
 [  330   586   999 ...     0     0     0]
 [   22   191   192 ...     0     0     0]]


In [ ]:
def bin

In [36]:
promedio_bin, pos_bin = times(bin_acot)
print(f"Tiempo promedio para el algoritmo binario acotado {promedio_bin}")
print(pos_bin)
promedio_sec, pos_sec = times(sec)
print(f"Tiempo promedio para el algoritmo de busqueda secuencial {promedio_sec}")
print(pos_sec)


Tiempo promedio para el algoritmo binario acotado 0.11066439151763915
[796, 452, 392, 134, 150, 213, 280, 63, 115, 54, 53, 24, 55, 98, 97, 112, 10, 45, 51, 54, 25, 0, 53, 20, 66, 53, 2, 16, 39, 38, 0, 27, 10, 3, 57, 16, 23, 2, 15, 55, 25, 5, 13, 37, 14, 16, 21, 14, 12, 43, 16, 20, 1, 17, 6, 4, 15, 14, 2, 1, 0, 14, 8, 12, 12, 1, 20, 41, 18, 5, 5, 0, 39, 12, 28, 15, 0, 14, 1, 9, 11, 3, 27, 13, 5, 25, 29, 1, 16, 41, 12, 14, 24, 7, 41, 18, 10, 4, 3, 14, 2173, 1132, 822, 397, 330, 582, 665, 143, 290, 89, 115, 34, 126, 371, 367, 360, 48, 66, 135, 86, 61, 0, 162, 35, 102, 161, 8, 25, 46, 98, 0, 77, 20, 5, 80, 51, 82, 10, 40, 56, 100, 16, 35, 128, 16, 44, 47, 64, 19, 167, 38, 48, 3, 38, 11, 7, 44, 27, 5, 1, 0, 30, 21, 40, 24, 4, 44, 88, 73, 12, 17, 2, 85, 25, 79, 28, 0, 37, 7, 25, 26, 7, 70, 21, 18, 69, 93, 3, 23, 82, 21, 43, 32, 19, 79, 46, 12, 7, 10, 17, 1158, 637, 515, 189, 192, 289, 410, 83, 155, 66, 68, 27, 71, 153, 154, 164, 11, 48, 66, 61, 40, 0, 79, 25, 77, 77, 4, 17, 40, 55, 0, 44, 13

In [46]:
promedio_bunb, pos_bunb = times(bin_unb)
print(f"Tiempo promedio para el algoritmo binario no acotado {promedio_bunb}")
print(pos_bunb)

Tiempo promedio para el algoritmo binario no acotado 0.04064292907714844
[796, 452, 392, 134, 150, 213, 280, 63, 115, 54, 53, 24, 55, 98, 97, 112, 10, 45, 51, 54, 25, 0, 53, 20, 66, 53, 2, 16, 39, 38, 0, 27, 10, 3, 57, 16, 23, 2, 15, 55, 25, 5, 13, 37, 14, 16, 21, 14, 12, 43, 16, 20, 1, 17, 6, 4, 15, 14, 2, 1, 0, 14, 8, 12, 12, 1, 20, 41, 18, 5, 5, 0, 39, 12, 28, 15, 0, 14, 1, 9, 11, 3, 27, 13, 5, 25, 29, 1, 16, 41, 12, 14, 24, 7, 41, 18, 10, 4, 3, 14, 2173, 1132, 822, 397, 330, 582, 665, 143, 290, 89, 115, 34, 126, 371, 367, 360, 48, 66, 135, 86, 61, 0, 162, 35, 102, 161, 8, 25, 46, 98, 0, 77, 20, 5, 80, 51, 82, 10, 40, 56, 100, 16, 35, 128, 16, 44, 47, 64, 19, 167, 38, 48, 3, 38, 11, 7, 44, 27, 5, 1, 0, 30, 21, 40, 24, 4, 44, 88, 73, 12, 17, 2, 85, 25, 79, 28, 0, 37, 7, 25, 26, 7, 70, 21, 18, 69, 93, 3, 23, 82, 21, 43, 32, 19, 79, 46, 12, 7, 10, 17, 1158, 637, 515, 189, 192, 289, 410, 83, 155, 66, 68, 27, 71, 153, 154, 164, 11, 48, 66, 61, 40, 0, 79, 25, 77, 77, 4, 17, 40, 55, 0, 44,